In [82]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options  
from time import sleep
import pymongo
import csv
import os   
import datetime


def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)


def parse_tweet(tweet):
    entry = {}
            
    #Get Time of Tweet
    entry['time'] = tweet.find("time")['datetime']


    # Gets Text of Tweet
    # L is the "layers" of the tweet.  L[2] is the text itself.
    Layers = []
    Layers.append(tweet.findAll("div",recursive=False)[1])
    Layers.append(Layers[0].findAll("div",recursive=False)[1])
    Layers.append(Layers[1].findAll("div",recursive=False)[len(Layers[1])-3].text)
    entry['text'] = Layers[2]


    #Get Handle
    entry['handle'] = Layers[0].findAll('a')[0]['href']


    #Get Is_Retweet
    info = tweet.find_parent('div').find('div').text
    entry['is_retweet'] = ('retweet' in info.lower())
    
    return entry

In [85]:
    

#Scrapes many tweets
def twitter(url,num_tweets):
    #SOURCE: https://medium.com/@dawranliou/twitter-scraper-tutorial-with-python-requests-beautifulsoup-and-selenium-part-2-b38d849b07fe
    chrome_options = Options()  
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"),   options=chrome_options)  
    driver.get(url)
    print('here')
    body = driver.find_element_by_tag_name('body')
    sleep(4)
    tweets = []
    for i in range(num_tweets):
        body.send_keys(Keys.PAGE_DOWN)
        html = driver.page_source
        soup = bs(html,'lxml')
        timeline = soup.findAll("div", {"data-testid" : "tweet"})
        
        for tweet in timeline:
            response = parse_tweet(tweet)
            if not (response in tweets):
                tweets.append(response)

    driver.close()
    return tweets

    

    
    
 

#     links = set()

#     for i in range(num_tweets):
#         body.send_keys(Keys.PAGE_DOWN)
#         mars_twitter_html = driver.page_source
#         soup = bs(mars_twitter_html,'lxml')
#         current_lines = pd.Series([x['href'] for x in soup.find_all(class_='css-4rbku5 css-18t94o4 css-901oao r-1re7ezh r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0')])
#         links.update(current_lines)

#     filepath = 'links_scraped.html'
#     with open(filepath,'w') as text:
#         for line in links:
#             try:
#                 text.write(str(line))
#                 text.write('\n')
#             except UnicodeEncodeError:
#                 text.write('line skipped')
#                 text.write('\n')
    
    
    
#     filepath2= 'all_tweets.html'
#     with open(filepath2,'w') as text:
#         for line in links:
#             response = tweet(f'https://twitter.com{line}')
#             for char in response:
#                 try:
#                     text.write(char)
#                 except UnicodeEncodeError:
#                     pass
#             text.write('\n')
    
   

In [86]:
for tweet in twitter('https://twitter.com/spear_jacob',10):
    print(tweet)

here
{'time': '2020-03-31T16:59:20.000Z', 'text': 'Um, so this isn’t my usual content but Kipo and the Age of Wonderbeasts on Netflix is one of the best shows I’ve ever watched.  I can’t recommend it highly enough!', 'handle': '/spear_jacob', 'is_retweet': False}
{'time': '2020-03-31T13:33:06.000Z', 'text': "You know what's not gonna reboot the economy? Austerity.\n\nYou know what they're gonna tell us our public services, including public universities, scientific grants, arts programs, &etc have to accept? Austerity.\n\nThe fight starts here, now.", 'handle': '/IBJIYONGI', 'is_retweet': True}
{'time': '2020-03-30T23:35:49.000Z', 'text': 'Undocumented labor sustains essential industries across the country. We are on the frontlines of feeding & protecting millions of Americans, yet we are excluded from the most basic protections & government support.\n\nIn the face of this pandemic, we must demand dignity & respect.', 'handle': '/CosechaMovement', 'is_retweet': True}
{'time': '2020-03-2

True